In [ ]:
import os

In [ ]:
%cd /home/jupyter-23523024/Eksperimen/amr-tst-indo

/home/jupyter-23523024/Eksperimen/amr-tst-indo


/opt/tljh/user/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
PROJECT_ROOT = "/home/jupyter-23523024/Eksperimen/amr-tst-indo"

In [ ]:
!git pull

Already up to date.


In [ ]:
# !pip install -r requirements.txt

# Initialization

## AMR Parsing

In [ ]:
from huggingface_hub import snapshot_download
# # Silver
# snapshot_download(
#     repo_id="abdiharyadi/indoamrbart-ft",
#     local_dir=f"{PROJECT_ROOT}/AMRBART-id/models",
#     allow_patterns="*mbart-en-id-smaller*",
#     ignore_patterns=[
#         "*log*",
#         "*iter*",
#     ]
# )

# Gold v2
amr_parsing_model_name = "mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan"
snapshot_download(
    repo_id=f"abdiharyadi/{amr_parsing_model_name}",
    local_dir=f"{PROJECT_ROOT}/AMRBART-id/models/{amr_parsing_model_name}",
    ignore_patterns=[
        "*log*",
        "*checkpoint*",
    ]
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/home/jupyter-23523024/Eksperimen/amr-tst-indo/AMRBART-id/models/mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan'

In [ ]:
from text_to_amr import TextToAMR
# t2a = TextToAMR(model_name="mbart-en-id-smaller-fted", use_prefix=True)  # Silver
t2a = TextToAMR(model_name=amr_parsing_model_name)  # Gold v2

/home/jupyter-23523024/.local/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
2025-03-07 14:24:20.850549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-07 14:24:20.878712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741332260.899811 1799410 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741332260.906246 1799410 cuda_

In [ ]:
import penman
graphs = t2a(["Halo, dunia!"])
print(penman.encode(graphs[0]))

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

# ::id 0
# ::annotator mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan
# ::snt Halo, dunia!
(z0 / dunia
    :mode ekspresif
    :mod (z1 / Halo
             :mode ekspresif))


## AMR-To-Text (AMR Generation)

### Using Nafkhan's method (old)

In [ ]:
# amr_gen_model_name = "mbart-en-id-smaller-fted-amr-generation-v2"  # Silver
# amr_gen_model_name = "mbart-en-id-smaller-indo-amr-generation-translated-nafkhan"  # Gold v2

In [ ]:
# from huggingface_hub import snapshot_download
# snapshot_download(
#     repo_id=f"abdiharyadi/{amr_gen_model_name}",
#     local_dir=f"{PROJECT_ROOT}/AMRBART-id/models/{amr_gen_model_name}",
#     ignore_patterns=[
#         "*checkpoint*"
#     ]
# )

In [ ]:
# from amr_to_text import AMRToText
# a2t = AMRToText(model_name=amr_gen_model_name)

### Using Taufiq's method (current)

In [ ]:
from amr_to_text import AMRToTextWithTaufiqMethod
amr_gen_model_name = "taufiq-indo-amr-generation-gold-v3.sen.join.cl-uncased"
model_path = f"./{amr_gen_model_name}"

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id=f"atikaistiqomah/{amr_gen_model_name}",
    local_dir=model_path,
    allow_patterns=[
        "*checkpoint-3*"
    ]
)

Fetching 88 files:   0%|          | 0/88 [00:00<?, ?it/s]

'/home/jupyter-23523024/Eksperimen/amr-tst-indo/taufiq-indo-amr-generation-gold-v3.sen.join.cl-uncased'

In [ ]:
a2t = AMRToTextWithTaufiqMethod(
    model_path=os.path.join(model_path, "checkpoint-3"),
    lowercase=True,
)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Running on the GPU


## AMR-TST

In [ ]:
from amr_tst_new import AMRTST
amr_tst = AMRTST(t2a=t2a, a2t=a2t)
amr_tst

# Normal inference
1. Edit `sentences`.
2. Edit `source_styles`. `LABEL_0` for negative, and `LABEL_1` for positive.
3. Execute.

In [ ]:
sentences = [
    "tolong admin untuk periksa pesan saya .",
    "kalian tidak suka belajar.",
    "kenapa saya tidak boleh pergi ?"
]
# source_styles = ["LABEL_0", "LABEL_0", "LABEL_0"], "LABEL_0", "LABEL_0"]
# assert len(sentences) == len(source_styles)
# assert len(sentences) == len(source_styles)

results, infos = amr_tst(sentences)
results

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 3/3 [00:00<00:00,  4.49it/s]


['tolong cek dm saya',
 'kalian ga suka belajar',
 'kenapa saya gabisa pergi ya ?']

In [ ]:
infos.to_list()[0]

{'source_text': 'tolong admin untuk periksa pesan saya .',
 'source_amr': '(z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / iksa-01 :ARG0 z1 :ARG1 (z3 / pesan-01 :ARG0 z1)))',
 'target_amr': '(z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / iksa-01 :ARG0 z1 :ARG1 (z3 / pesan-01 :ARG0 z1)))',
 'target_text': 'tolong cek dm saya'}

In [ ]:
# infos[4]

In [ ]:
for single_info in infos.to_list():
    x_src = single_info["source_text"]
    # s_src = single_info["source_style"]
    g_src_str = single_info["source_amr"]
    # a_src = single_info["style_words"]
    g_tgt_str = single_info["target_amr"]
    x_tgt = single_info["target_text"]

    print("--- [input] Teks sumber:", x_src)
    # print("--- [input] Gaya sumber:", s_src)
    print("--- Graf sumber:", g_src_str)
    # print("--- Kata bergaya:", a_src)
    print("--- Graf target:", g_tgt_str)
    print("--- [output] Teks target:", x_tgt)
    print("=======")

--- [input] Teks sumber: tolong admin untuk periksa pesan saya .
--- Graf sumber: (z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / iksa-01 :ARG0 z1 :ARG1 (z3 / pesan-01 :ARG0 z1)))
--- Graf target: (z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / iksa-01 :ARG0 z1 :ARG1 (z3 / pesan-01 :ARG0 z1)))
--- [output] Teks target: tolong cek dm saya
--- [input] Teks sumber: kalian tidak suka belajar.
--- Graf sumber: (z0 / suka-01 :polarity - :ARG0 (z1 / kalian) :ARG1 (z2 / belajar-01))
--- Graf target: (z0 / suka-01 :polarity - :ARG0 (z1 / kalian) :ARG1 (z2 / belajar-01))
--- [output] Teks target: kalian ga suka belajar
--- [input] Teks sumber: kenapa saya tidak boleh pergi ?
--- Graf sumber: (z0 / sebab-01 :ARG0 (z1 / amr-tidak-diketahui) :ARG1 (z2 / boleh-01 :polarity - :ARG1 (z3 / pergi-01 :ARG0 (z4 / aku))))
--- Graf target: (z0 / sebab-01 :ARG0 (z1 / amr-tidak-diketahui) :ARG1 (z2 / boleh-01 :polarity - :ARG1 (z3 / pergi-01 :ARG0 (z4 / aku))))
--- [output] Teks target: kenapa saya gabisa pergi ya 

# From file

## Without precomputed results, per data.

In [ ]:
filename0 = "/home/jupyter-23523024/Dataset/amr-tst-paralel/hasil_paralel_cl_formal-to-informal_0.txt"

In [ ]:
sentences = []
targets = []
source_styles = []  # Tetap disertakan jika model butuh informasi gaya sumber

with open(filename0, encoding="utf-8") as fp:
    for line in fp:
        line = line.strip()
        first_sentence, second_sentence = line.split("\t")  # Formal -> Informal
        sentences.append(first_sentence)
        source_styles.append("LABEL_0")  # Tetap berikan informasi bahwa input formal
        targets.append(second_sentence)

len(sentences), len(source_styles), len(targets)

(1000, 1000, 1000)

In [ ]:
from tqdm import tqdm
all_results = []
all_infos = []

In [ ]:
for i, (x, s, y) in enumerate(tqdm(zip(sentences, source_styles, targets), total=len(sentences))):
    if i < len(all_infos):
        continue

    results, infos = amr_tst([x], [s])
    all_results.extend(results)

    info = infos.to_list()[0]
    info["human_target_text"] = y
    all_infos.append(info)

len(all_infos)

  0%|          | 0/1000 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



100%|██████████| 1/1 [00:00<00:00, 78.87it/s]

100%|██████████| 1/1 [00:00<00:00, 8289.14it/s]



100%|██████████| 1000/1000 [41:05<00:00,  2.47s/it]


1000

In [ ]:
import json
with open("/home/jupyter-23523024/Eksperimen/amr_tst_formal-to-informal_result-eks 3-sentence-cl-join-gen-new-high(40).json", mode="w") as fp:
    json.dump(all_infos, fp, indent=2)

In [ ]:
from evaluation import compute_metrics_generation
preds = [x["target_text"] for x in all_infos]
references = [x["human_target_text"] for x in all_infos]

In [ ]:
# For BLEU-r
compute_metrics_generation(preds, references)

{'score': 10.954933108898516,
 'counts': [5797, 2072, 822, 352],
 'totals': [12209, 11209, 10212, 9217],
 'precisions': [47.48136620525842,
  18.48514586492997,
  8.049353701527615,
  3.8190300531626344],
 'bp': 0.8547609756443092,
 'sys_len': 12209,
 'ref_len': 14125}

In [ ]:
# For BLEU-s
compute_metrics_generation(preds, [x["source_text"] for x in all_infos])

{'score': 11.612124121442749,
 'counts': [6383, 2335, 956, 428],
 'totals': [12209, 11209, 10212, 9217],
 'precisions': [52.28110410353018,
  20.83147470782407,
  9.36153544849197,
  4.643593360095476],
 'bp': 0.7872182628119248,
 'sys_len': 12209,
 'ref_len': 15130}